# 1.3 OSMNx street network orientations

Adapted from the original notebook.

Author: [Geoff Boeing](https://geoffboeing.com/)

Compare the spatial orientations of city street networks with OSMnx.

  - [Documentation](https://osmnx.readthedocs.io/)
  - [Journal article and citation info](https://doi.org/10.1111/gean.70009)
  - [Code repository](https://github.com/gboeing/osmnx)
  - [Examples gallery](https://github.com/gboeing/osmnx-examples)

## 0. Imports and settings

In [ ]:
#!uv pip install --system --quiet osmnx[all]
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox

weight_by_length = False

ox.__version__

## 1. Geocode places

In [ ]:
# define the study sites as label : query
places = {
    'CABA': 'Ciudad Autónoma de Buenos Aires, Argentina',
}

In [ ]:
# verify OSMnx geocodes each query to what you expect (i.e., a [multi]polygon geometry)
gdf = ox.geocoder.geocode_to_gdf(list(places.values()))
gdf

In [ ]:
gdf.plot()

## 2. Iterate through places, creating graph, plotting

In [ ]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={"projection": "polar"})

# make sure axes is always a flat array
axes = np.atleast_1d(axes).ravel()

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    print(ox.utils.ts(), place)

    # get undirected graphs with edge bearing attributes
    G = ox.graph.graph_from_place(place, network_type="drive")
    Gu = ox.bearing.add_edge_bearings(ox.convert.to_undirected(G))
    fig, ax = ox.plot.plot_orientation(Gu, ax=ax, title=place, area=True)

# add figure title and save image
suptitle_font = {
    "family": "DejaVu Sans",
    "fontsize": 60,
    "fontweight": "normal",
    "y": 1,
}
fig.suptitle("City Street Network Orientation", **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig(f"../4_visualisation/{place}_street-orientations.png", facecolor="w", dpi=100, bbox_inches="tight")
plt.close()

## 3. Calculate entropy per city

You can also calculate the orientation entropy of a spatial graph with the `ox.bearing.orientation_entropy` function.

In [ ]:
ox.bearing.orientation_entropy(Gu)

## 4. Convert OSMnx graph to GeoDataFrames

In [ ]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(Gu, nodes=True, edges=True)

In [ ]:
edge_bearings = edges_gdf[['bearing', 'geometry']].copy()

In [ ]:
edge_bearings['bearing_180'] = edge_bearings['bearing'] % 180

In [ ]:
edge_bearings[['bearing', 'geometry']].plot(
    figsize=(12,12),
    column='bearing',
    legend=True
)

In [ ]:
edge_bearings.plot(
    figsize=(24,24),
    column='bearing_180',
    cmap='hsv',#'twilight_shifted',#
    legend=True
)

In [ ]:
edge_bearings

## 5. Save out streets with bearings

In [ ]:
edge_bearings.to_file(f'../0_data/in/{place}_edge_bearings.gpkg')